<a href="https://colab.research.google.com/github/Lorxus/Tontine/blob/main/tontine-deathcount-markovsim.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# numerical simulation for modeling daily death counts
# premise: each player has prob = 1/days+2 chance to die today and any day is like any other

import random as rand
import math

PLAYERCOUNT = 7141  # number of players in tontine, living and dead
LIVINGCOUNT = 280
DEATHSTHISYEAR = 32

def howmanygone(numdays: int, timespan: int) -> int:
    deathchance = 1/(numdays + 2)  # Laplace's Rule of Succession
    deathcount = 0
    currentlyalive = LIVINGCOUNT

    if numdays == 0:
        return 0
    for i in range(timespan):
        for j in range(currentlyalive):
            if rand.random() < deathchance:  # I know this isn't technically exactly 499 but this should be good enough in ensemble
                deathcount += 1
                currentlyalive -= 1

    return deathcount

def ensemble(numworlds: int, numdays: int, timespan: int) -> float:
    runoutcomes = [-1] * numworlds
    totaldeaths = 0
    for i in range(numworlds):
        thisrundeaths = howmanygone(numdays, timespan)
        runoutcomes[i] = thisrundeaths
        totaldeaths += thisrundeaths
        if rand.random() < 1/(numworlds ** 0.35):
                print('Run', i, '; deaths this time:', thisrundeaths)

    avgdeaths = (totaldeaths/numworlds)
    print('Average deaths:', avgdeaths)
    return avgdeaths

def ensemblethisyear(numworlds: int, numdays: int, timespan: int) -> float:
    runoutcomes = [-1] * numworlds
    totaldeaths = 0
    for i in range(numworlds):
        thisrundeaths = howmanygone(numdays, timespan)
        runoutcomes[i] = thisrundeaths
        totaldeaths += thisrundeaths
        if rand.random() < 1/(numworlds ** 0.35):
                print('Run', i, '; deaths this time:', DEATHSTHISYEAR + thisrundeaths)

    avgdeaths = DEATHSTHISYEAR + (totaldeaths/numworlds)
    print('Average deaths:', avgdeaths)
    return avgdeaths

print('How many days has it been since the start?')
dayslong = int(input())
print('How many days out are we predicting?')
ticktock = int(input())
print('How many ensemble runs?')
numworlds = int(input())
averagedeaths = ensemble(numworlds, dayslong, ticktock)


How many days has it been?
806
How many days out are we predicting?
287
How many ensemble runs?
1024
Run 22 ; deaths this time: 112
Run 25 ; deaths this time: 129
Run 44 ; deaths this time: 118
Run 51 ; deaths this time: 122
Run 52 ; deaths this time: 115
Run 64 ; deaths this time: 122
Run 71 ; deaths this time: 113
Run 90 ; deaths this time: 123
Run 96 ; deaths this time: 122
Run 104 ; deaths this time: 111
Run 125 ; deaths this time: 110
Run 126 ; deaths this time: 116
Run 140 ; deaths this time: 128
Run 144 ; deaths this time: 107
Run 194 ; deaths this time: 97
Run 196 ; deaths this time: 117
Run 210 ; deaths this time: 102
Run 216 ; deaths this time: 113
Run 236 ; deaths this time: 107
Run 269 ; deaths this time: 118
Run 296 ; deaths this time: 116
Run 297 ; deaths this time: 113
Run 320 ; deaths this time: 117
Run 321 ; deaths this time: 114
Run 328 ; deaths this time: 119
Run 337 ; deaths this time: 101
Run 340 ; deaths this time: 120
Run 348 ; deaths this time: 130
Run 363 ; dea